In [1]:
import geopandas as gp
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from pprint import pprint
import os
import glob
import openpyxl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.express as px #if using plotly
import folium
import warnings
import uuid
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.max_columns', None)  # display all columns
pd.options.display.float_format = '{:20,.2f}'.format  # suppress scientific notation

### RQ #2) Which specific flares are the most impactful? 

### RQ #3) Which specific block grous are being most impacted? 


Exploratory question; no hypothesis testing

EJ index = (The Environmental Indicator Percentile for Block Group) x (Demographic Index for Block Group)
  
a) Determine flaring impact metric. 

Something like: buffer_population x EJ Index x flare volume  

How best to normalize these different units? Use percentiles compared to rest of CA? 

b) Calculate impact per flare  

c) Visualize top_x flares by impact  


### Read in files

In [3]:
#ca_state = gp.read_file("data/CA_State_TIGER2016.shp")  # CA state
ca_counties = gp.read_file("data/CA_Counties_TIGER2016.shp")  # CA counties
ca_counties.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging

In [4]:
#ca_bg = gp.read_file("data/ca_bg_clean.shp")  # CA block groups, cleaned
# CA block groups merged with EJscreen data
ca_bg_joined = gp.read_file("data/ca_bg_joined_clean.shp")

# update col name for correct area b/c it gets saved as a truncated version when saved as a shapefile out of the 
# main data cleaning notebook
ca_bg_joined.rename(columns={'shape_ar_1':'shape_area_new'}, inplace=True)  # old:new.
ca_bg_joined.rename(columns={'CNTY_NAME':'cnty_name'}, inplace=True)  # old:new.
print(f'BGs found: {len(ca_bg_joined)}')

BGs found: 25607


In [5]:
# just CA flares
ca_flares = gp.read_file("data/ca_flares_clean.shp")

# renaming 
ca_flares.rename(columns={'NAMELSAD':'cnty_name'}, inplace=True)  # old:new. Match col names for merging

print(f'Flares found: {len(ca_flares)}')

Flares found: 117


In [6]:
ca_flares.geometry

0      POINT (101702.097 -392465.466)
1       POINT (50045.686 -404732.075)
2      POINT (111804.156 -403954.218)
3       POINT (67527.269 -411223.881)
4       POINT (62783.972 -410781.392)
                    ...              
112    POINT (165197.510 -469510.426)
113    POINT (165197.511 -469609.028)
114     POINT (-239947.045 35040.908)
115    POINT (245587.514 -437618.082)
116    POINT (225403.428 -439194.020)
Name: geometry, Length: 117, dtype: geometry

In [7]:
# set common crs for project
# epsg3310: https://epsg.io/3310-1739
# units: meters
meters_crs = 3310  # Projected crs. this should be good for this overlay() calculation and all of project. 

ca_counties = ca_counties.to_crs(meters_crs)
ca_flares = ca_flares.to_crs(meters_crs)
ca_bg_joined = ca_bg_joined.to_crs(meters_crs)

### Update ca_flares df

In [8]:
# not all flares have a unique identifier — both CatalogID and ID columns have missing values
# so I create a new col "flare_id" of IDs

# generate random integer IDs
ids = np.random.randint(100000, 999999, size=len(ca_flares))

# convert integer IDs to unique string identifiers
id_str = [str(uuid.uuid4())[:8] + str(i) for i in ids]

# add the new column to the DataFrame
ca_flares['flare_id'] = id_str

In [9]:
# set col list for BCM_avg calculation
bcm_list = ['BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021']

# add new column for average BCM across all years
ca_flares['BCM_avg'] = ca_flares[bcm_list].mean(axis=1) 

In [10]:
# subset to only columns needed for Tableau
col_list = ['flare_id','flare_cate','cnty_name','BCM_2012','BCM_2013','BCM_2014','BCM_2015','BCM_2016','BCM_2017',
            'BCM_2018','BCM_2019','BCM_2020','BCM_2021','BCM_avg', 'geometry']

ca_flares_sub = ca_flares[col_list].copy()

In [11]:
ca_flares_sub.sample(2)

,flare_id,flare_cate,cnty_name,BCM_2012,BCM_2013,BCM_2014,BCM_2015,BCM_2016,BCM_2017,BCM_2018,BCM_2019,BCM_2020,BCM_2021,BCM_avg,geometry
27,d4f9aeee267119,flares_upstream,Ventura County,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,0.00,POINT (53291.683 -406482.986)
81,3500b46e464101,flares_upstream,Fresno County,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,0.00,POINT (-17828.456 -209557.358)


### Update ca_bg_joined df

In [12]:
# There are five BGs that seem to just be aquatic buffers around the actual county land.
# Dropping them for now 
# Explore if needed: ca_bg_joined.explore()

ids_to_drop = [60839900000, 61119901000, 60379902000, 60379903000, 60599901000]

# Drop the rows with those IDs
ca_bg_joined = ca_bg_joined[~ca_bg_joined['bg_id'].isin(ids_to_drop)]

In [13]:
# subset to only cols needed
ca_bg_joined_sub = ca_bg_joined[['bg_id', 'cnty_name', 'ACSTOTPOP', 'MINORPOP',
                           'D_PM25_2', 'shape_area_new', 'geometry']].copy()

ca_bg_joined_sub.rename(columns={'geometry':'bg_geom'}, inplace=True)  # old:new. Match col names for merging

# counties_sub = ca_counties[['cnty_name', 'geometry']].copy()

# counties_sub.rename(columns={'geometry':'cnty_geom'}, inplace=True)  # old:new. Match col names for merging

# # add county names to social dataframe
# ca_bg_joined_sub = pd.merge(ca_bg_joined_sub, counties_sub, 
#                              on='cnty_name', how='left')

### Gather data by BG and export shapefile for Tableau

In [14]:


# social_df = social_df.set_geometry('bg_geom')

# #flares_df = set_geometry_buffer(flares_df, buffer_size)
# buffer_col = f"buffer_{buffer_size}m"
# flares_df[buffer_col] = flares_df['geometry'].buffer(distance=buffer_size)

# flares_df = flares_df.set_geometry(buffer_col)

# # temp = flares_df.unary_union
# # all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=flares_df.crs)  # convert back to geodf for processing

# test = social_df[social_df['bg_id'] == 60290033042].copy()

In [15]:
social_df = ca_bg_joined_sub
flares_df = ca_flares

In [16]:
social_df.sample(1)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,shape_area_new,bg_geom
18201,60952520001,Solano County,1598,465,28.20,"1,320,103.72","POLYGON ((-189817.820 4283.941, -189529.659 48..."


In [17]:
social_df = social_df.set_geometry('bg_geom')

social_df.geometry

0        POLYGON ((154828.423 -457797.384, 154927.488 -...
1        POLYGON ((154836.126 -458134.127, 154835.934 -...
2        POLYGON ((152442.207 -458985.139, 152441.728 -...
3        POLYGON ((152453.177 -459429.971, 152452.671 -...
4        POLYGON ((151858.050 -459765.962, 151869.518 -...
                               ...                        
25602    POLYGON ((-162287.592 -153450.003, -162255.599...
25603    POLYGON ((-153230.185 -157896.463, -153187.391...
25604    POLYGON ((-151685.856 -155314.179, -151680.464...
25605    POLYGON ((-151231.659 -155261.166, -151189.377...
25606    POLYGON ((52493.300 -162907.653, 52493.519 -16...
Name: bg_geom, Length: 25602, dtype: geometry

In [18]:
flares_df.rename(columns={'geometry':'flare_pts'}, inplace=True)  # old:new. Match col names for merging

In [19]:
#flares_df = set_geometry_buffer(flares_df, buffer_size)
buffer_size=2000
buffer_col = f"buffer_{buffer_size}m"
flares_df[buffer_col] = flares_df['flare_pts'].buffer(distance=buffer_size)

flares_df = flares_df.set_geometry('buffer_2000m')


In [20]:
# subset of desired columns from flares_df
#flares_df = flares_df.drop(['flare_pts'], axis=1)

flares_subset = flares_df[['buffer_2000m', 'BCM_avg', 'flare_id']]

# perform spatial join
intersect = gp.sjoin(social_df, flares_subset, how='left', predicate='intersects')


In [21]:
 # add county names to social dataframe
intersect_all = pd.merge(intersect, flares_subset[['flare_id', 'buffer_2000m']], 
                             on='flare_id', how='left')

In [22]:
len(intersect_all)

25865

In [29]:
# Check for null or invalid geometries in buffer_2000m column
intersect_all['buffer_valid'] = intersect_all['buffer_2000m'].is_valid

# Initialize int_prop and int_area columns with NaN values
intersect_all['int_prop'] = np.nan
intersect_all['int_area'] = np.nan

# Loop through each block group and flare pairing
for index, row in intersect_all.iterrows():
    # Check if buffer_2000m geometry is valid
    if row['buffer_valid']:
        # Calculate intersection area
        intersection = row['buffer_2000m'].intersection(row['bg_geom'])  # prop of buffer in the bg geom
        intersection_area = intersection.area
        # Calculate buffer area
        buffer_area = row['buffer_2000m'].area
        # Calculate intersection proportion
        buf_int_prop = intersection_area / buffer_area
        # Update int_prop and int_area columns with calculated values
        intersect_all.at[index, 'b_int_prop'] = buf_int_prop
        intersect_all.at[index, 'b_int_area'] = intersection_area
        intersect_all.at[index, 'buff_area'] = buffer_area

    else:
        # Assign 0 to int_prop and int_area if buffer_2000m geometry is invalid
        intersect_all.at[index, 'b_int_prop'] = 0
        intersect_all.at[index, 'b_int_area'] = 0
        intersect_all.at[index, 'buff_area'] = 0

        
# Drop buffer_valid column
intersect_all.drop('buffer_valid', axis=1, inplace=True)

# Create new dataframe to store output
#output_df = intersect_all[['bg_id', 'flare_id', 'int_prop', 'int_area']].copy()


In [30]:
len(intersect_all)

25865

In [33]:
intersect_all.sample(50)

,bg_id,cnty_name,ACSTOTPOP,MINORPOP,D_PM25_2,shape_area_new,bg_geom,index_right,BCM_avg,flare_id,buffer_2000m,int_prop,int_area,b_int_prop,b_int_area,buff_area
8044,60373025043,Los Angeles County,709,365,51.18,"74,863.61","POLYGON ((161218.992 -429822.340, 161228.730 -...",NaN,NaN,NaN,None,NaN,NaN,0.00,0.00,0.00
11505,60310012001,Kings County,1192,637,44.31,"250,933,380.29","POLYGON ((22898.361 -190878.242, 22920.694 -19...",NaN,NaN,NaN,None,NaN,NaN,0.00,0.00,0.00
25585,60590761052,Orange County,1925,1097,39.05,"522,364.13","POLYGON ((197542.586 -467424.926, 197542.494 -...",NaN,NaN,NaN,None,NaN,NaN,0.00,0.00,0.00
18343,60170308042,El Dorado County,750,0,4.70,"6,928,960.24","POLYGON ((-82150.324 69111.818, -82144.621 691...",NaN,NaN,NaN,None,NaN,NaN,0.00,0.00,0.00
23008,60290038151,Kern County,2876,1552,34.22,"1,212,043.12","POLYGON ((77540.890 -290459.707, 77539.564 -29...",NaN,NaN,NaN,None,NaN,NaN,0.00,0.00,0.00
8124,60990006021,Stanislaus County,1244,782,36.99,"2,081,468.59","POLYGON ((-92043.407 -38655.884, -91915.170 -3...",NaN,NaN,NaN,None,NaN,NaN,0.00,0.00,0.00
13176,60816129001,San Mateo County,1512,354,15.74,"596,995.04","POLYGON ((-195440.842 -62135.407, -195416.162 ...",NaN,NaN,NaN,None,NaN,NaN,0.00,0.00,0.00
10337,60375535021,Los Angeles County,1863,1408,54.19,"617,661.65","POLYGON ((170979.895 -455183.377, 171107.694 -...",NaN,NaN,NaN,None,NaN,NaN,0.00,0.00,0.00
379,60374033201,Los Angeles County,2586,2235,51.36,"2,329,213.02","POLYGON ((202272.768 -443580.541, 202296.836 -...",NaN,NaN,NaN,None,NaN,NaN,0.00,0.00,0.00
5766,60372088012,Los Angeles County,1075,923,63.87,"28,012.97","POLYGON ((158959.000 -437965.700, 159062.393 -...",NaN,NaN,NaN,None,NaN,NaN,0.00,0.00,0.00


In [36]:
test = intersect_all[intersect_all['bg_id'] == 60290033042][['bg_id', 'flare_id', 'b_int_prop', 'b_int_area', 'buff_area', 'bg_geom', 'buffer_2000m']]
#output_df.sample(10)

In [40]:
test

,bg_id,flare_id,b_int_prop,b_int_area,buff_area,bg_geom,buffer_2000m
17248,60290033042,ba9cd673447196,1.00,"12,546,193.96","12,546,193.96","POLYGON ((7651.644 -285527.714, 7653.631 -2855...","POLYGON ((48997.528 -304252.983, 48987.897 -30..."
17249,60290033042,c839d89d116793,1.00,"12,546,193.96","12,546,193.96","POLYGON ((7651.644 -285527.714, 7653.631 -2855...","POLYGON ((48965.526 -304151.318, 48955.895 -30..."
17250,60290033042,a7b294ec396160,1.00,"12,546,193.96","12,546,193.96","POLYGON ((7651.644 -285527.714, 7653.631 -2855...","POLYGON ((35062.910 -299372.866, 35053.279 -29..."
17251,60290033042,2711cfa1350000,1.00,"12,546,193.96","12,546,193.96","POLYGON ((7651.644 -285527.714, 7653.631 -2855...","POLYGON ((29674.254 -286900.381, 29664.624 -28..."
17252,60290033042,86368a7f526599,0.94,"11,811,996.58","12,546,193.96","POLYGON ((7651.644 -285527.714, 7653.631 -2855...","POLYGON ((34463.729 -285943.545, 34454.099 -28..."
17253,60290033042,a3617a3b394120,0.93,"11,662,288.94","12,546,193.96","POLYGON ((7651.644 -285527.714, 7653.631 -2855...","POLYGON ((34480.172 -285854.516, 34470.541 -28..."
17254,60290033042,4de56ee5838474,0.92,"11,563,001.65","12,546,193.96","POLYGON ((7651.644 -285527.714, 7653.631 -2855...","POLYGON ((34512.024 -285835.291, 34502.393 -28..."
17255,60290033042,d413694b983896,0.93,"11,643,318.51","12,546,193.96","POLYGON ((7651.644 -285527.714, 7653.631 -2855...","POLYGON ((34463.316 -285810.152, 34453.686 -28..."
17256,60290033042,ac8de52f523182,0.92,"11,596,848.08","12,546,193.96","POLYGON ((7651.644 -285527.714, 7653.631 -2855...","POLYGON ((34480.712 -285805.421, 34471.081 -28..."
17257,60290033042,a96e1040392205,0.92,"11,574,977.44","12,546,193.96","POLYGON ((7651.644 -285527.714, 7653.631 -2855...","POLYGON ((34485.032 -285796.519, 34475.401 -28..."


### I confirm this works as intended in the Folium mapping at the bottom of the notebook

In [ ]:
# Create new 'area' column for the areas of the intersections
intersect_final['intersect_area'] = intersect_final.area

# Calculate the proportion of each block group within the buffer zone
intersect_final['intersect_prop'] = intersect_final['intersect_area'] / intersect_final['shape_area_new']

In [ ]:
intersect_final[intersect_final['bg_id'] == 60290033042][['BCM_avg', 'bg_id', 'intersect_prop']]

In [ ]:
print(intersect_final['bg_id'].nunique())

In [ ]:
# Apply the proportion to each demographic variable to find counts by variable
demo_vars = ['ACSTOTPOP', 'MINORPOP']
for var in demo_vars:
    intersect_final[var + '_intersect_count'] = intersect_final[var] * intersect_final['intersect_prop']

# find overall proportions for each demo var by dividing var count by respective total population    
for var in demo_vars:
    intersect_final[var + '_bg_totprop'] = intersect_final[var] / intersect_final['ACSTOTPOP']

In [ ]:
# Define the variables to standardize
varlist = ['BCM_avg', 'D_PM25_2', 'ACSTOTPOP_intersect_count']

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(intersect_final[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_norm' for var in varlist])

# Concatenate the new variables with the original DataFrame
intersect_final = pd.concat([intersect_final, intersect_norm], axis=1)


In [ ]:
intersect_final.sample(5)

In [ ]:
# save shapefile for visualization in Tableau

#intersect_final = intersect[['flare_id', 'flare_cate', 'bg_id', 'cnty_name', 'shape_area_new', 'intersect_prop', 'BCM_avg', 'ACSTOTPOP_intersect_count', 'D_PM25_2', 'ACSTOTPOP_intersect_count_norm', 'BCM_avg_norm', 'D_PM25_2_norm', 'intersect_geom']].copy()

intersect_final.rename(columns={'shape_area_new':'shape_area'}, inplace=True)  # old:new.
intersect_final.rename(columns={'intersect_prop':'int_prop'}, inplace=True)  # old:new.
intersect_final.rename(columns={'ACSTOTPOP_intersect_count':'pop'}, inplace=True)  # old:new.
intersect_final.rename(columns={'ACSTOTPOP_intersect_count_norm':'pop_norm'}, inplace=True)  # old:new.
intersect_final.rename(columns={'BCM_avg_norm':'bcm_norm'}, inplace=True)  # old:new.
intersect_final.rename(columns={'D_PM25_2':'pm25'}, inplace=True)  # old:new.
intersect_final.rename(columns={'D_PM25_2_norm':'pm25_norm'}, inplace=True)  # old:new.
intersect_final.rename(columns={'intersect_geom':'int_geom'}, inplace=True)  # old:new.

# rename the flare categories for readability
a = (intersect_final["flare_cate"].
                                 replace({"flares_oil_downstream": "down_oil",
                                          "flares_upstream": "upstream"}))

intersect_final = intersect_final.set_geometry('int_geom')

In [ ]:
intersect_final.columns

In [ ]:
# Shapefiles can only have one geometry column
# drop the buffer geom and flare points as they're not needed for the BG-level impact analysis. 
bg_impact = intersect_final.drop(['buffer_2000m', 'geometry'], axis=1)

In [ ]:
bg_impact.sample(1)

In [ ]:
bcm_bg_avg = bg_impact.groupby('bg_id')['BCM_avg'].sum().reset_index(name='bcm_bg_avg')

test = pd.merge(bg_impact, bcm_bg_avg, on='bg_id', how='left')

In [ ]:
test[test['bg_id'] == 60290033042][['BCM_avg', 'bg_id', 'bcm_bg_avg', 'int_prop']]

In [ ]:
test[(test['bg_id'] == 60379800331) | (test['bg_id'] == 60290033042)][['BCM_avg', 'bg_id', 'bcm_bg_avg']].sample(5)


In [ ]:
bg_impact.to_file("data/df_bg_impactmetric_shp.shp", driver='ESRI Shapefile')

### Group by Flare ID and export shapefile for Tableau

### Create buffer

In [ ]:
def create_buffer_intersection(flares_df, social_df, buffer_size):
    """
    Creates a new Geodf containing the intersection between census BGs
    and buffers of {buffer size} around flare sites.
    
    Parameters
    ----------
    flares_df : A Geodf containing point geometries representing flare sites.
    social_df : A Geodf containing polygon geometries representing social data from EPA EJScreen
    buffer_size : int
        The size of the buffer around each flare site, in meters.
    
    Returns
    -------
    tuple of GeoDataFrames
        A tuple containing two GeoDataFrames:
        1. The original flares_df with a new buffer column added.
        2. A GeoDataFrame containing the intersection between social_df and the 
        union of all buffer geometries in flares_df.
    """
    
    social_df = social_df.set_geometry('bg_geom')
    
    #flares_df = set_geometry_buffer(flares_df, buffer_size)
    buffer_col = f"buffer_{buffer_size}m"
    flares_df[buffer_col] = flares_df['geometry'].buffer(distance=buffer_size)

    flares_df = flares_df.set_geometry(buffer_col)
    
    temp = flares_df.unary_union
    all_buffers = gp.GeoDataFrame({'geometry': [temp]}, crs=flares_df.crs)  # convert back to geodf for processing
    
    intersect_temp = gp.overlay(social_df, all_buffers, how='intersection')  # could look at keep_geom=False
    
    intersect_temp.rename(columns={'geometry':'intersect_geom'}, inplace=True)  # old:new. Match col names for merging
    
    intersect_temp = intersect_temp.set_geometry('intersect_geom')

    intersect_temp2 = gp.sjoin(intersect_temp, flares_df, how = "left", predicate = 'intersects')

    return flares_df, intersect_temp2

In [ ]:
buffer_sizes = [2000]
#buffer_sizes = [100, 400, 800, 1000, 1600, 2000, 5000, 7500, 10000]

for buffer_size in buffer_sizes:
    ca_flares_new, intersect = create_buffer_intersection(ca_flares_sub, ca_bg_joined_sub, buffer_size)

In [ ]:
ca_flares_new.geometry

In [ ]:
intersect_final.columns

In [ ]:
intersect_final.sample(1)

In [ ]:
# grab only necessary cols
intersect_forflares = intersect_final[['flare_id','bg_id', 'cnty_name_left', 'flare_cate',
                                 'BCM_2012', 'BCM_2013', 'BCM_2014', 'BCM_2015','BCM_2016', 
                                 'BCM_2017', 'BCM_2018', 'BCM_2019', 'BCM_2020', 'BCM_2021','BCM_avg',
                                 'buffer_2000m','pop','pm25','bcm_norm']].copy()

In [ ]:
# Define the variables to aggregate
agg_dict = {'pm25': 'mean', 'pop': 'sum'}

# Group the block groups by flare ID and calculate the aggregation for the variables
ca_flares_agg = intersect_forflares.groupby('flare_id').agg(agg_dict)

# Rename the output variables
ca_flares_agg = ca_flares_agg.rename(columns={'pm25': 'pm25_avg', 'pop': 'pop_sum'})

# Join the aggregated variables with the original columns
ca_flares_merged = intersect_forflares.merge(ca_flares_agg, on='flare_id')


In [ ]:
# Define the variables to standardize
varlist = ['pm25_avg', 'pop_sum']  # because bcm is already attached to the flare unit of analysis

# Create a StandardScaler object that will transform selected variables to have a mean of zero and 
# sd of 1.
scaler = StandardScaler()

# Fit and transform the selected variables
intersect_norm = scaler.fit_transform(ca_flares_merged[varlist])

# Create new variables with the standardized values
intersect_norm = pd.DataFrame(intersect_norm, columns=[var + '_n' for var in varlist])

# Concatenate the new variables with the original DataFrame
ca_flares_merged_f = pd.concat([ca_flares_merged, intersect_norm], axis=1)

In [ ]:
ca_flares_merged_f.sample(3)

In [ ]:
ca_flares_merged_f = gp.GeoDataFrame(ca_flares_merged_f, geometry='buffer_2000m', crs=meters_crs)


In [ ]:
ca_flares_merged_f.to_file("data/df_flare_impactmetric_shp.shp", driver='ESRI Shapefile')

## Calculating single instance of weights here in python

In [ ]:
# # Define the weights for each variable
# bcm_weight = 0
# pm25_weight = 1
# pop_weight = 0

In [ ]:
# # Calculate the weighted variables
# intersect['BCM_weighted'] = intersect['BCM_avg_norm'] * bcm_weight
# intersect['D_PM25_2_weighted'] = intersect['D_PM25_2_norm'] * pm25_weight
# intersect['ACSTOTPOP_weighted'] = intersect['ACSTOTPOP_intersect_count_norm'] * pop_weight

# # Define the variables to sum
# varlist_weighted = ['BCM_weighted', 'D_PM25_2_weighted', 'ACSTOTPOP_weighted']

# # Group the block groups by flare ID and sum the weighted variables
# ca_flares_merged = intersect.groupby('flare_id')[varlist_weighted].sum()

# # Calculate the impact metric as the sum of the weighted variables
# ca_flares_merged['impact_metric'] = ca_flares_merged[varlist_weighted].sum(axis=1)

# # Sort the flares by impact metric in descending order
# ca_flares_merged = ca_flares_merged.sort_values('impact_metric', ascending=False)

In [ ]:
# # Show the results
# ca_flares_merged.head()

In [ ]:
# # Define the variables to sum
# varlist_norm = ['BCM_avg_norm', 'D_PM25_2_norm', 'ACSTOTPOP_intersect_count_norm']

# # Group the block groups by flare ID and sum the standardized variables
# ca_flares_merged = intersect.groupby('flare_id')[varlist_norm].sum()

# # Calculate the impact metric as the sum of the standardized variables
# ca_flares_merged['impact_metric'] = ca_flares_merged[varlist_norm].sum(axis=1)

# # Sort the flares by impact metric in descending order
# ca_flares_merged = ca_flares_merged.sort_values('impact_metric', ascending=False)

In [ ]:
# df_final = pd.merge(ca_flares_merged, intersect, on='flare_id')

In [ ]:
# invalid_geoms = df_final[~df_final.is_valid]

In [ ]:
# invalid_geoms

In [ ]:
# # save for use in tableau
# df_final.to_csv(F"data/df_impactmetric_csv.csv", index=False)

In [ ]:
# # check for missing values
# missing_values = df_final.isnull().sum()

# # filter columns with missing values
# missing_cols = missing_values[missing_values > 0]

# # print column names and number of missing values
# for col in missing_cols.index:
#     print(f"Column '{col}' has {missing_cols[col]} missing values")

In [ ]:
# no_missing = missing_values[missing_values == 0]
# # print column names and number of missing values
# for col in no_missing.index:
#     print(f"Column '{col}' has {no_missing[col]} missing values")

In [ ]:
# df_final.sample(2)

## Top ten

In [ ]:
# # display the top ten flares by impact metric
# top_ten = for_map.nlargest(10, 'impact_metric')
# top_ten = top_ten.set_geometry('buffer_2000m')

In [ ]:
# # check for missing values
# missing_values = for_map.isnull().sum()

# # filter columns with missing values
# missing_cols = missing_values[missing_values > 0]

# # print column names and number of missing values
# for col in missing_cols.index:
#     print(f"Column '{col}' has {missing_cols[col]} missing values")

In [ ]:
# no_missing = missing_values[missing_values == 0]
# # print column names and number of missing values
# for col in no_missing.index:
#     print(f"Column '{col}' has {no_missing[col]} missing values")

## Folium Mapping

In [ ]:
# for_map = df_final[['flare_id', 'BCM_avg', 'D_PM25_2', 'ACSTOTPOP_intersect_count', 'impact_metric', 'buffer_2000m']]
# for_map = gp.GeoDataFrame(for_map, geometry='buffer_2000m', crs=meters_crs)

# for_map.to_file("data/df_impactmetric_shp.shp", driver='ESRI Shapefile')

In [43]:

# Define the color scale and number of bins
#color_scale = 'Reds'
#num_bins = 10

# Create a map centered on the first flare
# Create a folium map with a center location
m = folium.Map(location=[38.377158,-121.645792], zoom_start=6, tiles=None,overlay=False)  #start w lat/long roughly in center of CA
base_map = folium.FeatureGroup(name='Base map', overlay=True, control=False)
folium.TileLayer(tiles='OpenStreetMap').add_to(base_map)
base_map.add_to(m)




In [44]:
# Feature Group: Buffers

def style_function3(feature):
    return {
        'fillColor': 'red',
        'color': 'red',
        'fillOpacity': 0.05
    }

inter_all_bg = folium.FeatureGroup(name='Intersect BG', overlay=True)
folium.GeoJson(
    data=test["buffer_2000m"],
    style_function=style_function3
).add_to(inter_all_bg)
inter_all_bg.add_to(m)

In [45]:
# Feature Group: block group

def style_function2(feature):
    return {
        'fillColor': 'blue',
        'color': 'blue',
        'fillOpacity': 0.05
    }

bg_bound = folium.FeatureGroup(name='Intersect BG', overlay=True)
folium.GeoJson(
    data=test["bg_geom"],
    style_function=style_function2
).add_to(bg_bound)
bg_bound.add_to(m)

In [ ]:
# style_function = lambda x: {'fillColor': '#ffffff', 
#                             'color':'#000000', 
#                             'fillOpacity': 0.1, 
#                             'weight': 0.1}
# highlight_function = lambda x: {'fillColor': '#999999', 
#                                 'color':'#999999', 
#                                 'fillOpacity': 0.50, 
#                                 'weight': 0.1}
# NIL = folium.features.GeoJson(
#     data = for_map,
#     style_function=style_function, 
#     control=False,
#     highlight_function=highlight_function, 
#     tooltip=folium.features.GeoJsonTooltip(
#         fields=['flare_id', 'BCM_avg_norm', 'D_PM25_2_norm', 
#              'ACSTOTPOP_intersect_count_norm', 'impact_metric'],# 'D_PM25_2', 'ACSTOTPOP', 'MINORPOP','shape_area_new', 'intersect_prop', 'intersect_area', 'MINORPOP_bg_totprop'],
#         style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
#     )
# )

In [46]:
# # add hover functionality as child to map, add layering, display map
# m.add_child(NIL)
# m.keep_in_front(NIL)
# folium.LayerControl().add_to(m)

NameError: name 'NIL' is not defined

In [47]:
# Display the map
m